In [1]:
# QUESTION NUMBER 1
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from scipy.stats import norm
import numpy as np
import pandas as pd
import patsy
import os
import warnings
warnings.simplefilter('ignore')

In [5]:
data = pd.read_csv(r'C:\Users\Maricielo\Downloads\wage2015_subsample_inference.csv')
data = data.drop('rownames',axis=1)
list(data.columns)

['wage',
 'lwage',
 'sex',
 'shs',
 'hsg',
 'scl',
 'clg',
 'ad',
 'mw',
 'so',
 'we',
 'ne',
 'exp1',
 'exp2',
 'exp3',
 'exp4',
 'occ',
 'occ2',
 'ind',
 'ind2']

In [7]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Lasso
from statsmodels.api import add_constant


# Transformar variables categóricas a dummies (suponiendo que 'sex' y 'location' son categóricas)
# Convirtiendo 'sex' en una columna binaria (0 para 'female', 1 para 'male')
data['sex'] = data['sex'].apply(lambda x: 1 if x == 'male' else 0)

# Si hay otras variables categóricas, pueden transformarse utilizando OneHotEncoder o pandas.get_dummies
# Por ejemplo, transformar una columna 'location' con varias categorías:
# data = pd.get_dummies(data, columns=['location'], drop_first=True)

# Generar interacciones binarias
# Si necesitas calcular interacciones específicas entre variables, puedes hacerlo manualmente o programáticamente.
# Aquí se muestra un ejemplo para crear una columna de interacción entre 'treat' y 'sex' (si existe 'treat'):
if 'treat' in data.columns:
    data['treat_sex_interaction'] = data['treat'] * data['sex']

# Si necesitas un dataframe con todas las variables originales y las interacciones:
# Puedes agregar más interacciones de manera similar.

# Opcional: Agregar una constante (intercepto) si es necesario
data_with_const = add_constant(data)

print(data_with_const.head())


   const       wage     lwage  sex  shs  hsg  scl  clg   ad   mw  ...   we  \
0    1.0   9.615385  2.263364    0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0   
1    1.0  48.076923  3.872802    0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0   
2    1.0  11.057692  2.403126    0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0   
3    1.0  13.942308  2.634928    0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0   
4    1.0  28.846154  3.361977    0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0   

    ne  exp1  exp2    exp3     exp4     occ  occ2     ind  ind2  
0  1.0   7.0  0.49   0.343   0.2401  3600.0    11  8370.0    18  
1  1.0  31.0  9.61  29.791  92.3521  3050.0    10  5070.0     9  
2  1.0  18.0  3.24   5.832  10.4976  6260.0    19   770.0     4  
3  1.0  25.0  6.25  15.625  39.0625   420.0     1  6990.0    12  
4  1.0  22.0  4.84  10.648  23.4256  2015.0     6  9470.0    22  

[5 rows x 21 columns]


In [ ]:
# Variables categóricas identificadas
categorical_vars = ['sex', 'mw', 'so', 'we', 'ne']  # Género y ubicaciones

# Convertir variables categóricas en dummies
data = pd.get_dummies(data, columns=categorical_vars, drop_first=True)

# Generar todas las interacciones bidireccionales
interaction_terms = []
for col1 in data.columns:
    for col2 in data.columns:
        if col1 != col2 and f"{col2}*{col1}" not in interaction_terms:
            data[f"{col1}*{col2}"] = data[col1] * data[col2]
            interaction_terms.append(f"{col1}*{col2}")


In [ ]:
# Seleccionar la variable objetivo y las predictoras
y = data['wage']  # Salario como variable dependiente
X = data.drop(columns=['wage'])  # Resto de las variables como predictoras

# Escalamiento de las variables predictoras para Lasso
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
# Configuración de validación cruzada
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Ajustar modelo Lasso con selección de lambda óptima
lasso = LassoCV(cv=cv, random_state=42)
lasso.fit(X_scaled, y)

# Extraer coeficientes
lasso_coefficients = pd.Series(lasso.coef_, index=X.columns)

# Mostrar coeficientes seleccionados
print("Coeficientes seleccionados por Lasso:")
print(lasso_coefficients[lasso_coefficients != 0])


In [ ]:
# Reajuste del modelo con variables seleccionadas
selected_features = lasso_coefficients[lasso_coefficients != 0].index
X_selected = add_constant(X[selected_features])  # Agregar constante para la regresión

ols_model = OLS(y, X_selected).fit()

# Resumen del modelo
print(ols_model.summary())

# Interpretación: Analizar coeficientes
impact_clg = lasso_coefficients.filter(like='CLG')  # Filtros relacionados con CLG
print("\nImpacto de ser graduado universitario en diferentes grupos:")
print(impact_clg)


In [ ]:
pip install networkx matplotlib

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def draw_dag(edges, title):
    """
    Draws a Directed Acyclic Graph (DAG) based on the provided edges.
    
    Parameters:
    - edges: List of tuples representing directed edges (source, target)
    - title: Title of the graph
    """
    G = nx.DiGraph()
    G.add_edges_from(edges)

    plt.figure(figsize=(10, 6))
    pos = nx.spring_layout(G, seed=42)  # Layout for consistent placement
    nx.draw(G, pos, with_labels=True, node_size=3000, node_color="lightblue", font_size=10, font_weight="bold", arrowsize=15)
    plt.title(title)
    plt.show()

# DAG for the first scenario: Tabaquismo y función pulmonar
edges_tabaquismo = [
    ("Conducta individual de tabaquismo", "Volumen respiratorio forzado"),
    ("Edad", "Conducta individual de tabaquismo"),
    ("Edad", "Volumen respiratorio forzado"),
    ("Altura", "Volumen respiratorio forzado"),
    ("Sexo", "Conducta individual de tabaquismo"),
    ("Sexo", "Volumen respiratorio forzado")
]
draw_dag(edges_tabaquismo, "Efecto del Tabaquismo sobre la Función Pulmonar")

# DAG for the second scenario: Lactancia materna e infecciones
edges_lactancia = [
    ("Lactancia materna", "Número de infecciones del bebé"),
    ("Ingresos familiares", "Lactancia materna"),
    ("Ingresos familiares", "Número de infecciones del bebé"),
    ("Educación", "Lactancia materna"),
    ("Educación", "Número de infecciones del bebé"),
    ("Número de niños en el hogar", "Lactancia materna"),
    ("Número de niños en el hogar", "Número de infecciones del bebé"),
    ("Cuidado de niños fuera del hogar", "Número de infecciones del bebé"),
    ("Cuidado de niños fuera del hogar", "Lactancia materna"),
    ("Estado civil", "Lactancia materna"),
    ("Estado civil", "Número de infecciones del bebé")
]
draw_dag(edges_lactancia, "Efecto de la Lactancia Materna sobre las Infecciones")
